In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
from helper import *

Using gpu device 0: GeForce GT 630M (CNMeM is disabled)


In [2]:
#Normally construct the CNN, but keep and return the reference to the last convolutional layer
def build_cnn_with_conv(input_var=None):
    # Input layer
    network = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=input_var)

    # Convolutional layer with 32 kernels of size 5x5. Strided and padded
    # convolutions are supported as well; see the docstring.
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    # Max-pooling layer of factor 2 in both dimensions:
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # Another convolution with 32 5x5 kernels, and another 2x2 pooling:
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify)
    conv = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # A fully-connected layer of 256 units with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(conv, p=.5),
            num_units=256,
            nonlinearity=lasagne.nonlinearities.rectify)

    # And, finally, the 10-unit output layer with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)

    return network, conv #conv is the last conv layer

In [3]:
#Build and populate network
input_var = T.tensor4('input')
network, conv = build_cnn_with_conv(input_var)

#This network had 95% test accuracy on MNIST
with np.load('model.npz') as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]
lasagne.layers.set_all_param_values(network, param_values)

#Compile function to generate features
output_var = lasagne.layers.get_output(conv)
feat_extract = theano.function([input_var], output_var)

#Load dataset
train_dataset, train_labels, test_dataset, test_labels = load_not_mnist_dataset()

In [4]:
#Extract features, one by one so we dont explode memory
train_feats = np.empty((200000, 512), dtype='float32')
print("Loading training data")
for i in range(train_dataset.shape[0]):
    feats = feat_extract(train_dataset[i].reshape((1, 1, 28, 28)))
    train_feats[i] = np.array(feats.reshape((512)))
    
print("Loading test data")
test_feats = np.empty((test_dataset.shape[0], 512), dtype='float32')
for i in range(test_dataset.shape[0]):
    feats = feat_extract(test_dataset[i].reshape((1, 1, 28, 28)))
    test_feats[i] = np.array(feats.reshape((512)))



Loading training data
Loading test data


In [5]:
#Trains logistic regression on extracted features
import time
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
start_time = time.time()
log.fit(train_feats, train_labels)
print("Training took ", time.time() - start_time, " seconds")
print("Validation score: ", log.score(train_feats, train_labels))
print("Test score: ", log.score(test_feats, test_labels))


Training took  382.03640270233154  seconds
Validation score:  0.853575
Test score:  0.9053


In [6]:
#Trains linear regression on both extracted features and whole image for comparison
import time
from sklearn.linear_model import LinearRegression

print("Learned features:")
lin = LinearRegression()
lin.fit(train_feats, train_labels)
print("Validation score: ", lin.score(train_feats, train_labels))
print("Test score: ", lin.score(test_feats, test_labels))


#Train logistic regression with image as features for comparison
print("\nWhole image:")
train_dataraw = train_dataset.reshape((train_dataset.shape[0], 28 * 28))
test_dataraw = test_dataset.reshape((test_dataset.shape[0], 28 * 28))
lin_raw = LinearRegression()
lin_raw.fit(train_dataraw, train_labels)
print("Validation score: ", lin_raw.score(train_dataraw, train_labels))
print("Test score: ", lin_raw.score(test_dataraw, test_labels))


Learned features:
Validation score:  0.622372507308
Test score:  0.690627017155

Whole image:
Validation score:  0.552986870496
Test score:  0.641092281194
